In [ ]:
import csv
import json
import pandas as pd

In [ ]:
df = pd.read_csv("/content/total_dataset_utf8.csv")
print(len(df))

38410


In [ ]:
!pip install sentence_transformers
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 112.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 74.8 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=6be117d22ac5487039d5eafe128c4492924b7aea15f17cee87c8235c3a162b73
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence_transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import numpy as np
import itertools

from konlpy.tag import Okt,Kkma
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [ ]:
!pip install keybert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for keybert: filename=keybert-0.7.0-py3-none-any.whl size=23794 sha256=98eb05ba08a159928b8cd3743ab4aec0d5c7f2fa0a02afd35efd8e4edab2814f
  Stored in directory: /root/.cache/pip/wheels/68/aa/41/82025d89b0eb97484c9ac7d527abf596765c41733af79f86b0
Successfully built keybert


In [ ]:
tokenizer = Kkma()
n_gram_range = (2,3)
model = SentenceTransformer("sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens")
top_n = 5

In [ ]:
def extract_keybert(sentence,tokenizer,model, num = 5):
  tokenized_doc = tokenizer.pos(sentence)
  tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun'])
  n_gram_range = (2,3)
  count = CountVectorizer(ngram_range = n_gram_range).fit([tokenized_nouns])
  candidates = count.get_feature_names_out()


  doc_embedding = model.encode([sentence])
  candidate_embeddings = model.encode(candidates)

  distance = cosine_similarity(doc_embedding, candidate_embeddings)
  keywords = [candidates[index] for index in distance.argsort()[0][-num:]]
  return keywords




In [ ]:
from keybert import KeyBERT
kw_model = KeyBERT()

In [ ]:
def extract_keybert2(sentence,top_n= 5):
  #sentence = okt.normalize(sentence)
  if "_" in sentence:
    sentence = sentence.replace("_"," ")

  keywords_mmr = kw_model.extract_keywords(sentence,keyphrase_ngram_range=(2,3),use_mmr = True,top_n = 5,diversity = 0.2)
  result = []
  for i in range(len(keywords_mmr)):
    result.append(keywords_mmr[i][0])
  return result

In [ ]:
##월요일에 split -> okt.normalizer & okt.nouns로 변경해보기

def split_keybert(keybert):
  splited = []
  for word in keybert:
    splited.append(word.split())


  result = set([])
  for i in range (len(splited)):
    for j in range (len(splited[i])):
      word = splited[i][j]
      word = tokenizer.normalize(word)
      word = tokenizer.nouns(word)
      result.add(splited[i][j])


  
  result = list(result)

  return result

In [ ]:
def split_keybert_tokenizer(keybert):
  splited = []
  for sentence in keybert:
    #word = tokenizer.normalize(sentence)
    words = tokenizer.nouns(sentence)
    for w in words:
      if(len(w) > 1):
        splited.append(w)

  result = set(splited)
  result = list(result)
  return result

In [ ]:
def make_data(dataframe,index,model,tokenizer,top_n = 5):
  Basicdata = dict()
  Basicdata["title"] = df["title"][index]
  Basicdata["tag"] = df["tag"][index]
  Basicdata["desc"] = df["desc"][index]
  Basicdata["publisher"] = df["publisher"][index]
  Basicdata["date"] = df["modified_date"][index]
  Basicdata["accessURL"] = df["accessURL"][index]
  Basicdata["uid"] = int(df["uid"][index])
  sentence = Basicdata["title"] + " " + Basicdata["desc"]

  #keybert = extract_keybert(sentence,tokenizer,model)
  keybert = extract_keybert2(sentence)
  Basicdata["keybert"] = keybert
  keyword = split_keybert_tokenizer(keybert)
  Basicdata["keyword"] = keyword


  data = dict()
  data["basicMetadata"] = Basicdata
  data["extendedMetadata"] = []

  return data

**여기서부터 실행**

In [ ]:
Metadatalist = []
for i in range(len(df)):
  Metadatalist.append(make_data(df,i,model,tokenizer,top_n))
  print(i,"번쨰 작업을 완료했습니다")
  

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
33410 번쨰 작업을 완료했습니다
33411 번쨰 작업을 완료했습니다
33412 번쨰 작업을 완료했습니다
33413 번쨰 작업을 완료했습니다
33414 번쨰 작업을 완료했습니다
33415 번쨰 작업을 완료했습니다
33416 번쨰 작업을 완료했습니다
33417 번쨰 작업을 완료했습니다
33418 번쨰 작업을 완료했습니다
33419 번쨰 작업을 완료했습니다
33420 번쨰 작업을 완료했습니다
33421 번쨰 작업을 완료했습니다
33422 번쨰 작업을 완료했습니다
33423 번쨰 작업을 완료했습니다
33424 번쨰 작업을 완료했습니다
33425 번쨰 작업을 완료했습니다
33426 번쨰 작업을 완료했습니다
33427 번쨰 작업을 완료했습니다
33428 번쨰 작업을 완료했습니다
33429 번쨰 작업을 완료했습니다
33430 번쨰 작업을 완료했습니다
33431 번쨰 작업을 완료했습니다
33432 번쨰 작업을 완료했습니다
33433 번쨰 작업을 완료했습니다
33434 번쨰 작업을 완료했습니다
33435 번쨰 작업을 완료했습니다
33436 번쨰 작업을 완료했습니다
33437 번쨰 작업을 완료했습니다
33438 번쨰 작업을 완료했습니다
33439 번쨰 작업을 완료했습니다
33440 번쨰 작업을 완료했습니다
33441 번쨰 작업을 완료했습니다
33442 번쨰 작업을 완료했습니다
33443 번쨰 작업을 완료했습니다
33444 번쨰 작업을 완료했습니다
33445 번쨰 작업을 완료했습니다
33446 번쨰 작업을 완료했습니다
33447 번쨰 작업을 완료했습니다
33448 번쨰 작업을 완료했습니다
33449 번쨰 작업을 완료했습니다
33450 번쨰 작업을 완료했습니다
33451 번쨰 작업을 완료했습니다
33452 번쨰 작업을 완료했습니다
33453 번쨰 작업을 완료했습니다
33454 번쨰 작업을 완료했습니다
33455 번쨰 작업을 완료했습니다
33456 번쨰 작업을 완료했습니다
33457 번쨰 작업을 완료했습니다
3345

In [ ]:
json_output = json.dumps(Metadatalist,indent="\t",ensure_ascii = False)

with open("dataset_new.json","w") as outfile:
  outfile.write(json_output)

In [ ]:

'''
from google.colab import files
files.download("dataset_new.json")
'''

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

dictionary 처리

In [ ]:
with open("dataset_new.json", "r") as file:
  data = json.load(file)

In [ ]:
dlen = len(data)

In [ ]:
indexdict = dict()

for i in range(dlen):
  keyword = []
  for word in data[i]["basicMetadata"]["keyword"]:
    keyword.append(word)
    

  for word in keyword:
    if word in indexdict:
      indexdict[word].append(i)
    else:
      indexdict[word] = [i]
  print(i,"번째 작업 완료")

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
33410 번째 작업 완료
33411 번째 작업 완료
33412 번째 작업 완료
33413 번째 작업 완료
33414 번째 작업 완료
33415 번째 작업 완료
33416 번째 작업 완료
33417 번째 작업 완료
33418 번째 작업 완료
33419 번째 작업 완료
33420 번째 작업 완료
33421 번째 작업 완료
33422 번째 작업 완료
33423 번째 작업 완료
33424 번째 작업 완료
33425 번째 작업 완료
33426 번째 작업 완료
33427 번째 작업 완료
33428 번째 작업 완료
33429 번째 작업 완료
33430 번째 작업 완료
33431 번째 작업 완료
33432 번째 작업 완료
33433 번째 작업 완료
33434 번째 작업 완료
33435 번째 작업 완료
33436 번째 작업 완료
33437 번째 작업 완료
33438 번째 작업 완료
33439 번째 작업 완료
33440 번째 작업 완료
33441 번째 작업 완료
33442 번째 작업 완료
33443 번째 작업 완료
33444 번째 작업 완료
33445 번째 작업 완료
33446 번째 작업 완료
33447 번째 작업 완료
33448 번째 작업 완료
33449 번째 작업 완료
33450 번째 작업 완료
33451 번째 작업 완료
33452 번째 작업 완료
33453 번째 작업 완료
33454 번째 작업 완료
33455 번째 작업 완료
33456 번째 작업 완료
33457 번째 작업 완료
33458 번째 작업 완료
33459 번째 작업 완료
33460 번째 작업 완료
33461 번째 작업 완료
33462 번째 작업 완료
33463 번째 작업 완료
33464 번째 작업 완료
33465 번째 작업 완료
33466 번째 작업 완료
33467 번째 작업 완료
33468 번째 작업 완료
33469 번째 작업 완료
33470 번째 작업 완료
33471 번째 작업 완료
33472 번째 작업 완료
33473 번째 작업 완료
3347

In [ ]:
index_output = json.dumps(indexdict,indent="\t",ensure_ascii = False)

with open("index.json","w") as outfile:
  outfile.write(index_output)

In [ ]:
##files.download("index.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>